In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import requests
import zipfile
import glob
import pickle
import matplotlib.pyplot as plt

from parse_reference_spectra import parse_gnps_json, output_loop
from spectra_to_sirius import master_loop
from clone_ds_to_beta import copy_beta
from results_from_metaspace_msms_process import logon_metaspace
from metaspace.sm_annotation_utils import SMInstance
from results_from_metaspace_msms_process import split_data_frame_list
from sirius_to_metaspace_db import primary_loop
from cosine_spectra import score_alignment, read_ms_file
from metaspace_msms_mirror_spectra import mirror_main

Workflow steps on tissue data for METASAPCE MS/MS publication.

https://github.com/DinosaurInSpace/metaspace_msms_paper/blob/master/README.md
II. Steps 1-7

In [3]:
## 0. Load well behaved molecules from I.
# well_behaved_hits = read_pickle('intermediate/val_spotted.pickle')

# for testing, reality will be a subset of these:
well_behaved_hits = pd.read_csv('input/spotted_cmpds.csv', sep="\t")

In [4]:
## 1. List of ds_ids and metadata for selected experiments.
cmv3_wb_ds_name_dict = {'2016-09-21_16h07m45s': 'Technologie_ServierTES-WBrat-vehicle',
                        '2017-05-17_19h49m04s': 'whole body xenograft (1) [RMS norm]',
                        '2017-05-17_19h50m07s': 'wb xenograft trp pathway dosed- rms_corrected',
                        '2017-05-29_07h28m52s': 'servier_TT_mouse_wb_fmpts_derivatization_CHCA',
                        '2017-07-24_19h42m31s': 'Servier_Ctrl_mouse_wb_lateral_plane_9aa',
                        '2017-07-26_18h25m14s': 'Servier_Ctrl_mouse_wb_median_plane_9aa',
                        '2017-08-03_15h09m06s': 'Servier_Ctrl_mouse_wb_median_plane_chca',
                        '2017-08-03_15h09m51s': 'Servier_Ctrl_mouse_wb_lateral_plane_chca',
                        '2017-08-11_07h59m58s': 'Servier_Ctrl_mouse_wb_lateral_plane_DHB',
                        '2017-08-11_08h01m02s': 'Servier_Ctrl_mouse_wb_median_plane_DHB'
                        }

In [5]:
wholebody_ds_df = pd.DataFrame()
wholebody_ds_df['ds_id'] = cmv3_wb_ds_name_dict.keys()
wholebody_ds_df['polarity'] = ['positive', 'positive', 'positive', 
                               'positive', 'negative', 'negative', 
                               'positive', 'positive', 'positive', 
                               'positive']
wholebody_ds_df['group'] = ['Servier', 'Genentech', 'Genentech',
                           'Servier', 'Servier', 'Servier',
                           'Servier', 'Servier', 'Servier',
                           'Servier']
wholebody_ds_df['analyzer'] = 'FTICR'
wholebody_ds_df['expt_type'] = 'wholebody'
wholebody_ds_df['ds_id_in'] = wholebody_ds_df['ds_id']
wholebody_ds_df['ds_id_out'] = None
wholebody_ds_df['search'] = 'METASPACE_MSMS'
wholebody_ds_df['name'] = cmv3_wb_ds_name_dict.values()

wholebody_ds_df

,ds_id,polarity,group,analyzer,expt_type,ds_id_in,ds_id_out,search,name
0,2016-09-21_16h07m45s,positive,Servier,FTICR,wholebody,2016-09-21_16h07m45s,None,METASPACE_MSMS,Technologie_ServierTES-WBrat-vehicle
1,2017-05-17_19h49m04s,positive,Genentech,FTICR,wholebody,2017-05-17_19h49m04s,None,METASPACE_MSMS,whole body xenograft (1) [RMS norm]
2,2017-05-17_19h50m07s,positive,Genentech,FTICR,wholebody,2017-05-17_19h50m07s,None,METASPACE_MSMS,wb xenograft trp pathway dosed- rms_corrected
3,2017-05-29_07h28m52s,positive,Servier,FTICR,wholebody,2017-05-29_07h28m52s,None,METASPACE_MSMS,servier_TT_mouse_wb_fmpts_derivatization_CHCA
4,2017-07-24_19h42m31s,negative,Servier,FTICR,wholebody,2017-07-24_19h42m31s,None,METASPACE_MSMS,Servier_Ctrl_mouse_wb_lateral_plane_9aa
5,2017-07-26_18h25m14s,negative,Servier,FTICR,wholebody,2017-07-26_18h25m14s,None,METASPACE_MSMS,Servier_Ctrl_mouse_wb_median_plane_9aa
6,2017-08-03_15h09m06s,positive,Servier,FTICR,wholebody,2017-08-03_15h09m06s,None,METASPACE_MSMS,Servier_Ctrl_mouse_wb_median_plane_chca
7,2017-08-03_15h09m51s,positive,Servier,FTICR,wholebody,2017-08-03_15h09m51s,None,METASPACE_MSMS,Servier_Ctrl_mouse_wb_lateral_plane_chca
8,2017-08-11_07h59m58s,positive,Servier,FTICR,wholebody,2017-08-11_07h59m58s,None,METASPACE_MSMS,Servier_Ctrl_mouse_wb_lateral_plane_DHB
9,2017-08-11_08h01m02s,positive,Servier,FTICR,wholebody,2017-08-11_08h01m02s,None,METASPACE_MSMS,Servier_Ctrl_mouse_wb_median_plane_DHB


In [7]:
## 2. Download METASPACE results.
# Clones to beta and searches with cm3
out_dict_list = []
for ds in list(wholebody_ds_df.ds_id):
    x = copy_beta(ds, 'HNaKM', 'core_metabolome_v3')
    out_dict_list.append(x)
    
out_df = pd.DataFrame(out_dict_list)
out_df.to_pickle('intermediate/wholebody_cm3_df.pickle')

{'ds_id_in': '2016-09-21_16h07m45s', 'ds_id_out': '2020-06-26_22h10m03s'}
{'ds_id_in': '2017-05-17_19h49m04s', 'ds_id_out': '2020-06-26_22h10m06s'}
{'ds_id_in': '2017-05-17_19h50m07s', 'ds_id_out': '2020-06-26_22h10m09s'}
{'ds_id_in': '2017-05-29_07h28m52s', 'ds_id_out': '2020-06-26_22h10m12s'}
{'ds_id_in': '2017-07-24_19h42m31s', 'ds_id_out': '2020-06-26_22h10m16s'}
{'ds_id_in': '2017-07-26_18h25m14s', 'ds_id_out': '2020-06-26_22h10m19s'}
{'ds_id_in': '2017-08-03_15h09m06s', 'ds_id_out': '2020-06-26_22h10m22s'}
{'ds_id_in': '2017-08-03_15h09m51s', 'ds_id_out': '2020-06-26_22h10m25s'}
{'ds_id_in': '2017-08-11_07h59m58s', 'ds_id_out': '2020-06-26_22h10m28s'}
{'ds_id_in': '2017-08-11_08h01m02s', 'ds_id_out': '2020-06-26_22h10m31s'}


In [ ]:
# Waiting on jobs to complete!

In [8]:
# Downloads results
df_list = []
for ms_ds_id in out_df.ds_id_out:
    ds_id_in = list(out_df[out_df.ds_id_out == ms_ds_id].ds_id_in)[0]
    sm = SMInstance(host='https://beta.metaspace2020.eu')
    sm = logon_metaspace(sm)
    ds = sm.dataset(id=ms_ds_id)
    results_df = ds.results(database='core_metabolome_v3').reset_index()
    results_df['ds_id_in'] = ds_id_in
    df_list.append(results_df)

In [9]:
# Filters MS1 results for FDR <=0.2
ms1_df = pd.concat(df_list)
ms1_df = ms1_df[ms1_df.fdr <=0.2]
ms1_df = ms1_df[['ds_id_in', 'moleculeIds']]
ms1_df = split_data_frame_list(ms1_df, 'moleculeIds')

In [11]:
## 3. Generating custom MS2 database using observed compounds.
for ds_id in list(wholebody_ds_df.ds_id):
    limit_list = list(ms1_df[ms1_df.ds_id_in == ds_id].moleculeIds.unique())
    polarity = list(wholebody_ds_df[wholebody_ds_df.ds_id == ds_id].polarity)[0]
    primary_loop(limit_list,
                 ds,
                 'II_wholebody',
                 polarity,
                 'intermediate/exp_positive.pickle',
                 'intermediate/theo_positive.pickle',
                 'intermediate/exp_negative.pickle',
                 'intermediate/theo_negative.pickle',
                 'intermediate/ref_expt_df.pickle',
                 'intermediate/ref_theo_df.pickle'
                 )

# Send databases to Vitaly for uploading

2 sirius_output_df (9941, 3)                                                                                                exists  \
0_HMDB0003361_Pyrimidine                            trees/positive_0_HMDB0003361_Pyrimidine/0_HMDB...   
5_HMDB0029713_Thiazole                              trees/positive_5_HMDB0029713_Thiazole/5_HMDB00...   
17_HMDB0001414_Putrescine                           trees/positive_17_HMDB0001414_Putrescine/17_HM...   
18_HMDB0000056_betaAlanine                          trees/positive_18_HMDB0000056_betaAlanine/18_H...   
19_HMDB0000161_LAlanine                             trees/positive_19_HMDB0000161_LAlanine/19_HMDB...   
...                                                                                               ...   
11431_HMDB0058988_CL2045Z8Z11Z14Z1802045Z8Z11Z1...  trees/positive_11431_HMDB0058988_CL2045Z8Z11Z1...   
11432_HMDB0057261_CL1802045Z8Z11Z14Z2045Z8Z11Z1...  trees/positive_11432_HMDB0057261_CL1802045Z8Z1...   
11433_HMDB0059079_CL2045Z8

ValueError: Invalid file path or buffer object type: <class 'pandas.core.series.Series'>

In [ ]:
## 4. Running and interpreting the METASPACE MSMS results.
from clone_ds_to_beta import copy_beta

ori_ds_db = ['2016-09-21_16h07m45s',
             '2017-05-17_19h49m04s',
             '2017-05-17_19h50m07s',
             '2017-05-29_07h28m52s',
             '2017-07-24_19h42m31s',
             '2017-07-26_18h25m14s',
             '2017-08-03_15h09m06s',
             '2017-08-03_15h09m51s',
             '2017-08-11_07h59m58s',
             '2017-08-11_08h01m02s']
out_dict_list = []
for ds in ori_ds_db:
    print(ds)
    x = copy_beta(ds, 'M', ds)
    out_dict_list.append(x)
    
out_df = pd.DataFrame(out_dict_list)
out_df.to_pickle('intermediate/wholebody_ms2_df.pickle')

In [ ]:
wholebody_ds_df.update(out_df)

In [ ]:
## 5. Generate colocalization weighted psuedo-MS/MS spectra from ISF data.

In [ ]:
# Test inputs, replace with actual data when available.
original_ds_id_on_prod = '2020-03-12_17h55m21s'  # Update to run!
db_id_on_beta = '2020-05-13_17h50m21s'  # Update to run!
ds_id_on_beta = '2020-05-14_16h32m01s'  # Update to run!
path_to_reports = 'TEMP/reporting/'  # Update to run!
polarity = ['positive', 'negative'][0]  # Update to run!
psuedo_y_axis = ['binary', 'fdr', 'msm', 'cos', 'intensity'][0]
ref_spectra_df = pd.read_pickle("input/cm3_reference_spectra_df.pickle")

In [ ]:
# Real inputs as data becomes available
original_ds_id_on_prod = None  # Update to run!
db_id_on_beta = None  # Update to run!
ds_id_on_beta = None  # Update to run!
path_to_reports = 'TEMP/reporting/'  # Update to run!
polarity = ['positive', 'negative'][0]  # Update to run!
psuedo_y_axis = ['binary', 'fdr', 'msm', 'cos', 'intensity'][0]
ref_spectra_df = pd.read_pickle("input/cm3_reference_spectra_df.pickle")


In [ ]:
# Filter reference spectra and results on good spotted from above!
x = list(spotted_cmpds_df.id)
ref_spectra_df = ref_spectra_df[ref_spectra_df.id.isin(x)]

In [ ]:
# Have to run some kind of loop for each dataset!
mirror_main(original_ds_id_on_prod,
                db_id_on_beta,
                ds_id_on_beta,
                path_to_reports,
                polarity,
                psuedo_y_axis,
                ref_spectra_df)

In [ ]:
## 6. Score predicted, experimental, and psuedo-MS/MS spectra together for well-behaved subset.

# What scoring method?

In [ ]:
## 7. Plot examples.
# Make a list of good examples to plot for paper...
def load_arr_pickle(path):
    # Loads pickled np.array
    with open(path, 'rb') as f:
        return pickle.load(f)

plt.imshow(load_arr_pickle('*.pickle'))

To do:
1. Run through it and check to make sure steps work.